In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder

In [2]:
################################################
#                   FUNCTION                   #
################################################

################## CHECK DATA ##################
na_train = set()
def check_na(df,na_set):
    na_set.clear()
    for col_name in df:
        xna = len(df[df[col_name] == 'XNA'])
        na = df[col_name].isnull().sum()
        amt = df['SK_ID_CURR'].count()
        missing = xna + na
        if missing > 0:
            print('\n%s:  %d - %1.2f%% \n' %(col_name, missing, missing/amt),end='-'*40)
            if missing/amt >= 0.3:
                na_set.add(col_name + '*')
            else: na_set.add(col_name)


################## ADJUST ##################
#  Delete object which has missing values in some features
def na_filter_row(col,df):
    amt_mis_1 = df[col].isnull().sum() + len(df[df[col] == 'XNA'])
    print('\n Before deal: %s have %d missing values' %( col, amt_mis_1) )
    print('Processing...')
    df_notna = df.drop(df.loc[df[col].isnull()].index)
    df_notmissing = df_notna.drop(df_notna.loc[df_notna[col]=='XNA'].index)
    amt_mis_2 = len(df_notmissing[df_notmissing[col] == 'XNA']) + df_notmissing[col].isnull().sum()
    print('After deal: %s have %d missing values \n' %(col, amt_mis_2 ), end='-'*60 )
    return df_notmissing

In [3]:
################################################
#               DEAL WITH NA-VALUES            #
################################################
application_train = pd.read_csv('application_train.csv')
train_df = application_train.copy()
check_na(train_df,na_train)


CODE_GENDER:  4 - 0.00% 
----------------------------------------
AMT_ANNUITY:  12 - 0.00% 
----------------------------------------
AMT_GOODS_PRICE:  278 - 0.00% 
----------------------------------------
NAME_TYPE_SUITE:  1292 - 0.00% 
----------------------------------------
OWN_CAR_AGE:  202929 - 0.66% 
----------------------------------------
OCCUPATION_TYPE:  96391 - 0.31% 
----------------------------------------
CNT_FAM_MEMBERS:  2 - 0.00% 
----------------------------------------
ORGANIZATION_TYPE:  55374 - 0.18% 
----------------------------------------
EXT_SOURCE_1:  173378 - 0.56% 
----------------------------------------
EXT_SOURCE_2:  660 - 0.00% 
----------------------------------------
EXT_SOURCE_3:  60965 - 0.20% 
----------------------------------------
APARTMENTS_AVG:  156061 - 0.51% 
----------------------------------------
BASEMENTAREA_AVG:  179943 - 0.59% 
----------------------------------------
YEARS_BEGINEXPLUATATION_AVG:  150007 - 0.49% 
----------------------

In [4]:
################# Special part #################

for i in train_df:
    if i.endswith('AVG') or i.endswith('MODE') or i.endswith('MEDI'):
        # not raw, we can't handle, we don't know how they are calculated (noise, outlier)
        del train_df[i]

    elif i.startswith('EXT'):
        train_df[i].fillna(0.5,inplace = True) # 0.5 is not bad, but not good

    elif i.endswith('CIRCLE'):
        del train_df[i] # don't have infor about how to calculate "social circle"

    elif i in {'NAME_TYPE_SUITE','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START'}:
        del train_df[i] # Feature selection ( delete feature we will not use)

check_na(train_df,na_train) # require to check


CODE_GENDER:  4 - 0.00% 
----------------------------------------
AMT_ANNUITY:  12 - 0.00% 
----------------------------------------
AMT_GOODS_PRICE:  278 - 0.00% 
----------------------------------------
OWN_CAR_AGE:  202929 - 0.66% 
----------------------------------------
OCCUPATION_TYPE:  96391 - 0.31% 
----------------------------------------
CNT_FAM_MEMBERS:  2 - 0.00% 
----------------------------------------
ORGANIZATION_TYPE:  55374 - 0.18% 
----------------------------------------
DAYS_LAST_PHONE_CHANGE:  1 - 0.00% 
----------------------------------------
AMT_REQ_CREDIT_BUREAU_HOUR:  41519 - 0.14% 
----------------------------------------
AMT_REQ_CREDIT_BUREAU_DAY:  41519 - 0.14% 
----------------------------------------
AMT_REQ_CREDIT_BUREAU_WEEK:  41519 - 0.14% 
----------------------------------------
AMT_REQ_CREDIT_BUREAU_MON:  41519 - 0.14% 
----------------------------------------
AMT_REQ_CREDIT_BUREAU_QRT:  41519 - 0.14% 
----------------------------------------
AMT_

In [5]:
################## Delete Na-observation ##################

for col in na_train:
    if col.endswith('*'): del train_df[col[:-1]] #  delete feature miss >= 30% data
    else: train_df = na_filter_row(col,train_df) # with feature miss < 30%, we delete Na obs


 Before deal: CODE_GENDER have 4 missing values
Processing...
After deal: CODE_GENDER have 0 missing values 
------------------------------------------------------------
 Before deal: ORGANIZATION_TYPE have 55374 missing values
Processing...
After deal: ORGANIZATION_TYPE have 0 missing values 
------------------------------------------------------------
 Before deal: AMT_REQ_CREDIT_BUREAU_QRT have 33856 missing values
Processing...
After deal: AMT_REQ_CREDIT_BUREAU_QRT have 0 missing values 
------------------------------------------------------------
 Before deal: AMT_ANNUITY have 11 missing values
Processing...
After deal: AMT_ANNUITY have 0 missing values 
------------------------------------------------------------
 Before deal: DAYS_LAST_PHONE_CHANGE have 0 missing values
Processing...
After deal: DAYS_LAST_PHONE_CHANGE have 0 missing values 
------------------------------------------------------------
 Before deal: CNT_FAM_MEMBERS have 1 missing values
Processing...
After deal: 

In [6]:
check_na(train_df,na_train)

In [7]:
################## SUM OF SAME FEATURES ##################

train_df['SUM_FLAG_DOCUMENT'] = train_df.loc[:, 'FLAG_DOCUMENT_2':'FLAG_DOCUMENT_20'].sum(axis=1)
train_df['SUM_AMT_REQ'] = train_df.loc[:, 'AMT_REQ_CREDIT_BUREAU_HOUR':'AMT_REQ_CREDIT_BUREAU_YEAR'].sum(axis=1)
# del unnecessary col
for i in train_df.columns.tolist():
    if i.startswith('FLAG_DOC') or i.startswith('AMT_REQ'): del train_df[i]

# Export not null file
train_df.to_csv('application_train_notnull.csv', index=False)

In [8]:
'''
train_notnull = pd.read_csv('application_train_notnull.csv')
################################################
#              DEAL WITH OUTLIER              #
################################################

################## QUANTILE CUTOFF MENTHOD ##################

cutoff = 0.99
drop_idx = []
for column, dtype, in zip(train_notnull.columns, train_notnull.dtypes):
    if dtype == "float64":
        drop_idx += list(train_notnull[train_notnull[column]>train_notnull[column].quantile(cutoff)].index)
    if dtype == "int64" and train_notnull[column].unique().shape[0]>10:
        drop_idx += list(train_notnull[train_notnull[column]>train_notnull[column].quantile(cutoff)].index)
drop_idx = list(dict.fromkeys(drop_idx))
train_notnull.drop(drop_idx, inplace=True)

# Range of age in Vietnam from 20-60, so we need to do some adjustment
train_notnull = train_notnull[train_notnull.DAYS_BIRTH >= (-61*365)]

# check the repayment difficulties percent
mean_def = train_notnull['TARGET'].mean()
print ("%.2f%% of the loans in the training data have repayment difficulties." %(mean_def*100))
print('Training Features shape: ', train_notnull.shape)
train_notnull.to_csv('application_train_clean.csv', index=False)
'''

'\ntrain_notnull = pd.read_csv(\'application_train_notnull.csv\')\n################################################\n#              DEAL WITH OUTLIER              #\n################################################\n\n################## QUANTILE CUTOFF MENTHOD ##################\n\ncutoff = 0.99\ndrop_idx = []\nfor column, dtype, in zip(train_notnull.columns, train_notnull.dtypes):\n    if dtype == "float64":\n        drop_idx += list(train_notnull[train_notnull[column]>train_notnull[column].quantile(cutoff)].index)\n    if dtype == "int64" and train_notnull[column].unique().shape[0]>10:\n        drop_idx += list(train_notnull[train_notnull[column]>train_notnull[column].quantile(cutoff)].index)\ndrop_idx = list(dict.fromkeys(drop_idx))\ntrain_notnull.drop(drop_idx, inplace=True)\n\n# Range of age in Vietnam from 20-60, so we need to do some adjustment\ntrain_notnull = train_notnull[train_notnull.DAYS_BIRTH >= (-61*365)]\n\n# check the repayment difficulties percent\nmean_def = train_no

In [9]:
################################################
#                   ENCODING                   #
################################################

#train_df_encd = pd.read_csv('application_train_clean.csv') # disable if use LOF
train_df_encd = pd.read_csv('application_train_notnull.csv') # For LOF

# Number of each type of column
print('Data type of column: \n', train_df_encd.dtypes.value_counts())
# Number of unique classes in each object column
print('\n Unique classes in each object column: \n',train_df_encd.select_dtypes('object').apply(pd.Series.nunique, axis = 0))

# Encoding Categorical Variables
le = LabelEncoder()
le_count_train = 0

for col in train_df_encd:
    if train_df_encd[col].dtype == 'object':
        if len(list(train_df_encd[col].unique())) <= 2:
            le.fit(train_df_encd[col])
            train_df_encd[col] = le.transform(train_df_encd[col])
            le_count_train += 1
print('\n%d columns were label encoded\n' %le_count_train)

# [ORGANIZATION TYPE] synthesis
train_df_encd['ORGANIZATION_TYPE'] = train_df_encd['ORGANIZATION_TYPE'].map(lambda x: x.split(':')[0] )
train_df_encd['ORGANIZATION_TYPE'] = train_df_encd['ORGANIZATION_TYPE'].map(lambda x: x.split(' Type')[0] )

# One-hot endcoding
train_df_encd = pd.get_dummies(train_df_encd)

# check the repayment difficulties percent
mean_def = train_df_encd['TARGET'].mean()
print ("%.2f%% of the loans in the training data have repayment difficulties." %(mean_def*100))
print('Training Features shape: ', train_df_encd.shape)
train_df_encd.to_csv('application_train_encd.csv', index=False)

Data type of column: 
 int64      21
float64    12
object      9
dtype: int64

 Unique classes in each object column: 
 NAME_CONTRACT_TYPE      2
CODE_GENDER             2
FLAG_OWN_CAR            2
FLAG_OWN_REALTY         2
NAME_INCOME_TYPE        7
NAME_EDUCATION_TYPE     5
NAME_FAMILY_STATUS      5
NAME_HOUSING_TYPE       6
ORGANIZATION_TYPE      57
dtype: int64

4 columns were label encoded

8.28% of the loans in the training data have repayment difficulties.
Training Features shape:  (218049, 94)


In [10]:
# Another method to detect outlier ( time: 17')
# Nếu dùng pp này thì cần encoding trước và xóa phương pháp loại trên kia đi

################## LOCAL OUTLIER FACTOR ##################

id = train_df_encd['SK_ID_CURR'].values
del train_df_encd['SK_ID_CURR']
y = train_df_encd['TARGET'].values
del train_df_encd['TARGET']
X = train_df_encd.values

# identify outliers in the training dataset
lof = LocalOutlierFactor()
y_pred = lof.fit_predict(X)

# select all rows that are not outliers
out_id = y_pred != -1
X, y, id = X[out_id, :], y[out_id], id[out_id]

# summarize the shape of the updated training dataset
print(X.shape, y.shape, id.shape)
train_df_encd.columns.tolist()
train_v2 = pd.DataFrame(X, columns = train_df_encd.columns.tolist())
train_v2['TARGET'] = y
train_v2['SK_ID_CURR'] = id
train_v2 = train_v2[train_v2.DAYS_BIRTH >= (-61*365)]
train_v2 = train_v2[train_v2.CNT_CHILDREN <= 3]

mean_def = train_v2['TARGET'].mean()
print ("%.2f%% of the loans in the training data have repayment difficulties." %(mean_def*100))

(208228, 92) (208228,) (208228,)
8.37% of the loans in the training data have repayment difficulties.


In [11]:
# check the repayment difficulties percent

#mean_def = train_df_encd['TARGET'].mean()   # disable if use LOF
mean_def = train_v2['TARGET'].mean()       # For LOF
print ("%.2f%% of the loans in the training data have repayment difficulties." %(mean_def*100))

#train_df_encd.to_csv('application_train_final_quantile.csv', index=False)    # disable if use LOF
train_v2.to_csv('application_train_final_lof.csv', index=False)        # For LOF

8.37% of the loans in the training data have repayment difficulties.
